In [ ]:
'''
You are given an array containing specific HTTP error codes to analyze. You also have a log dataset that includes a mix of:

error events
non-error events (warnings, info logs)
successful requests
events from multiple services

Your task is to:
Filter events where status_code is present in the given error codes
Count how many times each error code occurs
Return the top-K most frequent error codes

Given:
error_codes = [404,403,429,500,503]
k = 3
Input Schema:
Column Name	Type
timestamp	String
service	String
user_id	Integer
status	String
status_code	Integer
Example Rows:
timestamp	service	user_id	status	status_code
2025-01-10 10:01:12	auth	101	error	404
2025-01-10 10:01:14	auth	102	error	404
2025-01-10 10:01:20	payments	201	error	500
2025-01-10 10:01:25	orders	305	error	403
2025-01-10 10:01:50	orders	401	success	200
2025-01-10 10:02:03	profile	501	success	200
2025-01-10 10:02:10	auth	103	error	404
2025-01-10 10:02:22	payments	204	warning	300
2025-01-10 10:02:35	orders	308	error	403
Output Schema:
Column Name	Type
error_code	Integer
error_count	Long
Example Output (For k = 2):
error_code	error_count
403	4
404	3
While solving, consider k=3

Starter Code
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

# Given inputs
error_codes = [404,403,429,500,503]
k = 3

data = [
    ("2025-01-10 10:01:12", "auth",     101, "error",   404),
    ("2025-01-10 10:01:14", "auth",     102, "error",   404),
    ("2025-01-10 10:01:15", "auth",     103, "error",   404),
    ("2025-01-10 10:01:20", "payments", 201, "error",   500),
    ("2025-01-10 10:01:23", "payments", 202, "error",   500),
    ("2025-01-10 10:01:25", "orders",   305, "error",   403),
    ("2025-01-10 10:01:28", "orders",   306, "error",   403),
    ("2025-01-10 10:01:32", "orders",   307, "error",   403),
    ("2025-01-10 10:01:35", "orders",   308, "error",   403),
    ("2025-01-10 10:01:20", "payments", 201, "error",   429),
    ("2025-01-10 10:01:21", "payments", 201, "error",   429),
    ("2025-01-10 10:01:22", "payments", 201, "error",   429),
    ("2025-01-10 10:01:23", "payments", 201, "error",   429),
    ("2025-01-10 10:01:24", "payments", 201, "error",   429),
    ("2025-01-10 10:01:24", "products", 201, "error",   503),
    ("2025-01-10 10:04:24", "products", 311, "error",   503),
    ("2025-01-10 10:07:24", "products", 128, "error",   503),
    ("2025-01-10 10:06:24", "products", 900, "error",   503),
    ("2025-01-10 10:10:24", "products", 201, "error",   503),
    ("2025-01-10 10:15:24", "products", 234, "error",   503),
    ("2025-01-10 10:18:24", "products", 111, "error",   503),
    ("2025-01-10 10:19:24", "products", 201, "error",   503),
    # SUCCESS rows with proper 2xx HTTP codes
    ("2025-01-10 10:01:50", "orders",   401, "success", 200),
    ("2025-01-10 10:02:40", "auth",     160, "success", 200),
    ("2025-01-10 10:02:45", "orders",   409, "success", 200),
    ("2025-01-10 10:02:52", "payments", 215, "success", 200),
    ("2025-01-10 10:03:01", "profile",  510, "success", 200),
    ("2025-01-10 10:03:12", "auth",     161, "success", 200),
    ("2025-01-10 10:03:20", "orders",   412, "success", 200),
    ("2025-01-10 10:03:28", "payments", 218, "success", 200),
    ("2025-01-10 10:03:35", "profile",  520, "success", 200),
    ("2025-01-10 10:03:48", "auth",     165, "success", 200),
    ("2025-01-10 10:03:55", "orders",   430, "success", 200),
    ("2025-01-10 10:02:03", "profile",  501, "success", 201),
    # Warning/info rows with 3xx codes
    ("2025-01-10 10:02:22", "payments", 204, "warning", 300),
    ("2025-01-10 10:02:30", "auth",     150, "info",    304),
]

input_df = spark.createDataFrame(
    data,
    ["timestamp", "service", "user_id", "status", "status_code"]
)

# TODO: Write your solution here

display(df_result)

'''

# Initialize Spark session
from pyspark.sql import SparkSession, functions as F
spark = SparkSession.builder.appName('Spark Playground').getOrCreate()

# Given inputs
error_codes = [404,403,429,500,503]
k = 3

data = [
    ("2025-01-10 10:01:12", "auth",     101, "error",   404),
    ("2025-01-10 10:01:14", "auth",     102, "error",   404),
    ("2025-01-10 10:01:15", "auth",     103, "error",   404),
    ("2025-01-10 10:01:20", "payments", 201, "error",   500),
    ("2025-01-10 10:01:23", "payments", 202, "error",   500),
    ("2025-01-10 10:01:25", "orders",   305, "error",   403),
    ("2025-01-10 10:01:28", "orders",   306, "error",   403),
    ("2025-01-10 10:01:32", "orders",   307, "error",   403),
    ("2025-01-10 10:01:35", "orders",   308, "error",   403),
    ("2025-01-10 10:01:20", "payments", 201, "error",   429),
    ("2025-01-10 10:01:21", "payments", 201, "error",   429),
    ("2025-01-10 10:01:22", "payments", 201, "error",   429),
    ("2025-01-10 10:01:23", "payments", 201, "error",   429),
    ("2025-01-10 10:01:24", "payments", 201, "error",   429),
    ("2025-01-10 10:01:24", "products", 201, "error",   503),
    ("2025-01-10 10:04:24", "products", 311, "error",   503),
    ("2025-01-10 10:07:24", "products", 128, "error",   503),
    ("2025-01-10 10:06:24", "products", 900, "error",   503),
    ("2025-01-10 10:10:24", "products", 201, "error",   503),
    ("2025-01-10 10:15:24", "products", 234, "error",   503),
    ("2025-01-10 10:18:24", "products", 111, "error",   503),
    ("2025-01-10 10:19:24", "products", 201, "error",   503),
    # SUCCESS rows with proper 2xx HTTP codes
    ("2025-01-10 10:01:50", "orders",   401, "success", 200),
    ("2025-01-10 10:02:40", "auth",     160, "success", 200),
    ("2025-01-10 10:02:45", "orders",   409, "success", 200),
    ("2025-01-10 10:02:52", "payments", 215, "success", 200),
    ("2025-01-10 10:03:01", "profile",  510, "success", 200),
    ("2025-01-10 10:03:12", "auth",     161, "success", 200),
    ("2025-01-10 10:03:20", "orders",   412, "success", 200),
    ("2025-01-10 10:03:28", "payments", 218, "success", 200),
    ("2025-01-10 10:03:35", "profile",  520, "success", 200),
    ("2025-01-10 10:03:48", "auth",     165, "success", 200),
    ("2025-01-10 10:03:55", "orders",   430, "success", 200),
    ("2025-01-10 10:02:03", "profile",  501, "success", 201),
    # Warning/info rows with 3xx codes
    ("2025-01-10 10:02:22", "payments", 204, "warning", 300),
    ("2025-01-10 10:02:30", "auth",     150, "info",    304),
]

input_df = spark.createDataFrame(
    data,
    ["timestamp", "service", "user_id", "status", "status_code"]
)

df_result = (
  input_df.filter(F.col("status_code").isin(error_codes)) #Filter events where status_code is present in the given error codes
  .withColumnRenamed("status_code", "error_code") # Rename column to match output schema
  .groupBy("error_code")
  .agg(F.count("*").alias("error_count")) # Count how many times each error code occurs
  .orderBy(F.col("error_count").desc())
  .limit(k) # limit to top-K most frequent error codes
)

# Display result
df_result)